# Importando o dataset...

In [1]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import os

kagglehub.login()


In [2]:
titanic_path = kagglehub.competition_download('titanic')

print('Data source import complete.')


Data source import complete.


In [3]:
train_csv_path = os.path.join(titanic_path, "train.csv")
test_csv_path = os.path.join(titanic_path, "test.csv")
df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

passenger_ids = df_test['PassengerId']

***Dataset* antes do pré-processamento:**

In [4]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Pré-processamento dos dados

Inicialmente, irei retirar colunas indesejadas, além de trocar valores nulos pela média geral daquele atributo. Assim, caso não saibamos se alguém tinha irmãos a bordo, por exemplo, iremos "chutar" a partir da média geral dos outros passageiros.

In [5]:
df_train = df_train.drop(columns = ["Ticket", "PassengerId", "Name"])
df_test = df_test.drop(columns = ["Ticket", "Name"])

# Substituir NaN pela média nas colunas "SibSp" e "Parch"
df_train.fillna(df_train["SibSp"].mean(), inplace=True)
df_train.fillna(df_train["Parch"].mean(), inplace=True)

df_test.fillna(df_test["SibSp"].mean(), inplace=True)
df_test.fillna(df_test["Parch"].mean(), inplace=True)

Depois, irei unificar os dados envolvendo a quantidade de parentes que cada náufrago tinha a bordo do navio em uma só coluna e excluir as colunas que geraram essa coluna unificada.

In [6]:
# Nova coluna com a soma de todos os parentes no navio
df_train["FamilyAboard"] = df_train["SibSp"] + df_train["Parch"]
df_test["FamilyAboard"] = df_test["SibSp"] + df_test["Parch"]

# Exclui as colunas usadas para formar a tabela anteriormente citada
df_train = df_train.drop(columns = ["SibSp", "Parch"])
df_test = df_test.drop(columns = ["SibSp", "Parch"])

Agora irei fazer a codificação de variáveis categóricas, a fim de facilitar meu trabalho ao montar o modelo preditivo.

In [7]:
# One-hot encoding
df_train = pd.get_dummies(df_train, columns=['Embarked', 'Cabin', 'Pclass'], drop_first=True)
df_train['Sex'] = df_train['Sex'].map({'male': 0, 'female': 1})

df_test = pd.get_dummies(df_test, columns=['Embarked', 'Cabin', 'Pclass'], drop_first=True)
df_test['Sex'] = df_test['Sex'].map({'male': 0, 'female': 1})

Para garantir que o modelo Random Forest (que será utilizado a seguir) funcione de maneira eficaz, é essencial realizar a normalização dos dados numéricos e o codificação das variáveis categóricas. Isso evita que variáveis com escalas maiores, como "Fare", influenciem indevidamente o desempenho do modelo em comparação com variáveis menores, como "Age".

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_train[['Age', 'Fare']] = scaler.fit_transform(df_train[['Age', 'Fare']])

df_test[['Age', 'Fare']] = scaler.transform(df_test[['Age', 'Fare']])

***Dataset* após o pré-processamento:**

In [9]:
df_train.head()

,Survived,Sex,Age,Fare,FamilyAboard,Embarked_C,Embarked_Q,Embarked_S,Cabin_A10,Cabin_A14,...,Cabin_F G63,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Pclass_2,Pclass_3
0,0,0,0.271174,0.014151,1,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,1,0.472229,0.139136,1,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1,1,0.321438,0.015469,0,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1,1,0.434531,0.103644,1,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0,0,0.434531,0.015713,0,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True


# Configuração do modelo...

Primeiramente, vamos escolher o tamanho do nosso teste interno, bem como o quão randômica será essa divisão.

In [10]:
X = df_train.drop(columns=['Survived'])
y = df_train['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [12]:
y_pred = model.predict(X_val)
print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

0.8100558659217877
              precision    recall  f1-score   support

           0       0.80      0.90      0.85       105
           1       0.83      0.68      0.75        74

    accuracy                           0.81       179
   macro avg       0.82      0.79      0.80       179
weighted avg       0.81      0.81      0.81       179



In [13]:
# Reindexar df_test sem PassengerId
df_test = df_test.reindex(columns=df_train.columns.drop('Survived').tolist(), fill_value=0)

y_test_pred = model.predict(df_test)

# Adicionar PassengerId de volta
df_test['PassengerId'] = passenger_ids

# Criação do DataFrame de submissão
submission_df = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],
    'Survived': y_test_pred
})

submission_df.to_csv('submission.csv', index=False)